# Rekurentné neurónové siete

V predchádzajúcom module sme používali bohaté sémantické reprezentácie textu a jednoduchý lineárny klasifikátor nad embeddingmi. Táto architektúra zachytáva agregovaný význam slov vo vete, ale nezohľadňuje **poradie** slov, pretože operácia agregácie nad embeddingmi túto informáciu z pôvodného textu odstránila. Keďže tieto modely nedokážu modelovať poradie slov, nemôžu riešiť zložitejšie alebo nejednoznačné úlohy, ako je generovanie textu alebo odpovedanie na otázky.

Na zachytenie významu textovej sekvencie potrebujeme použiť inú architektúru neurónovej siete, ktorá sa nazýva **rekurentná neurónová sieť** (RNN). V RNN prechádzame vetou cez sieť po jednom symbole a sieť produkuje určitý **stav**, ktorý následne odovzdáme s ďalším symbolom opäť do siete.

Daná vstupná sekvencia tokenov $X_0,\dots,X_n$, RNN vytvára sekvenciu blokov neurónovej siete a trénuje túto sekvenciu end-to-end pomocou spätného šírenia. Každý blok siete prijíma dvojicu $(X_i,S_i)$ ako vstup a produkuje $S_{i+1}$ ako výsledok. Konečný stav $S_n$ alebo výstup $X_n$ sa odovzdáva do lineárneho klasifikátora na produkciu výsledku. Všetky bloky siete zdieľajú rovnaké váhy a trénujú sa end-to-end jedným priechodom spätného šírenia.

Keďže stavové vektory $S_0,\dots,S_n$ prechádzajú sieťou, táto dokáže učiť sekvenčné závislosti medzi slovami. Napríklad, keď sa niekde v sekvencii objaví slovo *not*, sieť sa môže naučiť negovať určité prvky v stavovom vektore, čo vedie k negácii.

> Keďže váhy všetkých blokov RNN na obrázku sú zdieľané, ten istý obrázok môže byť reprezentovaný ako jeden blok (vpravo) s rekurentnou spätnou väzbou, ktorá odovzdáva výstupný stav siete späť na vstup.

Pozrime sa, ako nám rekurentné neurónové siete môžu pomôcť klasifikovať našu dátovú sadu správ.


In [1]:
import torch
import torchtext
from torchnlp import *
train_dataset, test_dataset, classes, vocab = load_dataset()
vocab_size = len(vocab)

Loading dataset...
Building vocab...


## Jednoduchý RNN klasifikátor

V prípade jednoduchého RNN je každá rekurentná jednotka jednoduchá lineárna sieť, ktorá prijíma zreťazený vstupný vektor a stavový vektor a vytvára nový stavový vektor. PyTorch reprezentuje túto jednotku pomocou triedy `RNNCell` a sieť takýchto buniek ako vrstvu `RNN`.

Na definovanie RNN klasifikátora najskôr použijeme vrstvu embedding na zníženie dimenzionality vstupnej slovnej zásoby a potom na ňu pridáme vrstvu RNN:


In [2]:
class RNNClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_class):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.embedding = torch.nn.Embedding(vocab_size, embed_dim)
        self.rnn = torch.nn.RNN(embed_dim,hidden_dim,batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, num_class)

    def forward(self, x):
        batch_size = x.size(0)
        x = self.embedding(x)
        x,h = self.rnn(x)
        return self.fc(x.mean(dim=1))

> **Poznámka:** V tomto prípade používame nevytrénovanú embedding vrstvu pre jednoduchosť, ale pre ešte lepšie výsledky môžeme použiť predtrénovanú embedding vrstvu s Word2Vec alebo GloVe embeddingami, ako bolo popísané v predchádzajúcej jednotke. Pre lepšie pochopenie by ste mohli tento kód prispôsobiť tak, aby pracoval s predtrénovanými embeddingami.

V našom prípade použijeme dávkovač s vypĺňaním (padded data loader), takže každá dávka bude obsahovať určitý počet sekvencií doplnených na rovnakú dĺžku. RNN vrstva prijme sekvenciu embedding tensorov a vygeneruje dva výstupy:
* $x$ je sekvencia výstupov RNN buniek na každom kroku
* $h$ je konečný skrytý stav pre posledný prvok sekvencie

Následne aplikujeme plne prepojený lineárny klasifikátor, aby sme získali počet tried.

> **Poznámka:** RNN sú pomerne náročné na trénovanie, pretože keď sa RNN bunky rozvinú pozdĺž dĺžky sekvencie, výsledný počet vrstiev zapojených do spätného šírenia je pomerne veľký. Preto je potrebné zvoliť malú rýchlosť učenia a trénovať sieť na väčšej množine údajov, aby sa dosiahli dobré výsledky. Môže to trvať pomerne dlho, takže použitie GPU je preferované.


In [3]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=padify, shuffle=True)
net = RNNClassifier(vocab_size,64,32,len(classes)).to(device)
train_epoch(net,train_loader, lr=0.001)

3200: acc=0.3090625
6400: acc=0.38921875
9600: acc=0.4590625
12800: acc=0.511953125
16000: acc=0.5506875
19200: acc=0.57921875
22400: acc=0.6070089285714285
25600: acc=0.6304296875
28800: acc=0.6484027777777778
32000: acc=0.66509375
35200: acc=0.6790056818181818
38400: acc=0.6929166666666666
41600: acc=0.7035817307692308
44800: acc=0.7137276785714286
48000: acc=0.72225
51200: acc=0.73001953125
54400: acc=0.7372794117647059
57600: acc=0.7436631944444444
60800: acc=0.7503947368421052
64000: acc=0.75634375
67200: acc=0.7615773809523809
70400: acc=0.7662642045454545
73600: acc=0.7708423913043478
76800: acc=0.7751822916666666
80000: acc=0.7790625
83200: acc=0.7825
86400: acc=0.7858564814814815
89600: acc=0.7890513392857142
92800: acc=0.7920474137931034
96000: acc=0.7952708333333334
99200: acc=0.7982258064516129
102400: acc=0.80099609375
105600: acc=0.8037594696969697
108800: acc=0.8060569852941176


## Long Short Term Memory (LSTM)

Jedným z hlavných problémov klasických RNN je takzvaný problém **miznúcich gradientov**. Keďže RNN sa trénujú end-to-end v jednom priechode spätného šírenia, majú problém prenášať chybu do prvých vrstiev siete, a preto sa sieť nedokáže naučiť vzťahy medzi vzdialenými tokenmi. Jedným zo spôsobov, ako sa tomuto problému vyhnúť, je zavedenie **explicitného riadenia stavu** pomocou takzvaných **brán**. Existujú dve najznámejšie architektúry tohto druhu: **Long Short Term Memory** (LSTM) a **Gated Relay Unit** (GRU).

![Obrázok znázorňujúci príklad bunky Long Short Term Memory](../../../../../lessons/5-NLP/16-RNN/images/long-short-term-memory-cell.svg)

LSTM sieť je organizovaná podobne ako RNN, ale existujú dva stavy, ktoré sa prenášajú z vrstvy do vrstvy: aktuálny stav $c$ a skrytý vektor $h$. V každej jednotke sa skrytý vektor $h_i$ spája so vstupom $x_i$, a spolu riadia, čo sa deje so stavom $c$ prostredníctvom **brán**. Každá brána je neurónová sieť so sigmoidovou aktiváciou (výstup v rozsahu $[0,1]$), ktorú si môžeme predstaviť ako bitovú masku, keď sa vynásobí stavovým vektorom. Existujú nasledujúce brány (zľava doprava na obrázku vyššie):
* **forget gate** (brána zabúdania) berie skrytý vektor a určuje, ktoré komponenty vektora $c$ potrebujeme zabudnúť a ktoré preniesť ďalej.
* **input gate** (vstupná brána) berie niektoré informácie zo vstupu a skrytého vektora a vkladá ich do stavu.
* **output gate** (výstupná brána) transformuje stav cez nejakú lineárnu vrstvu s $\tanh$ aktiváciou, potom vyberá niektoré z jeho komponentov pomocou skrytého vektora $h_i$, aby vytvorila nový stav $c_{i+1}$.

Komponenty stavu $c$ si môžeme predstaviť ako nejaké príznaky, ktoré môžeme zapínať a vypínať. Napríklad, keď v sekvencii narazíme na meno *Alice*, môžeme predpokladať, že ide o ženskú postavu, a nastaviť príznak v stave, že vety obsahujú ženské podstatné meno. Keď ďalej narazíme na frázu *and Tom*, nastavíme príznak, že máme množné číslo. Takto môžeme manipuláciou so stavom údajne sledovať gramatické vlastnosti častí vety.

> **Note**: Skvelým zdrojom na pochopenie vnútorných štruktúr LSTM je tento výborný článok [Understanding LSTM Networks](https://colah.github.io/posts/2015-08-Understanding-LSTMs/) od Christophera Olaha.

Aj keď vnútorná štruktúra LSTM bunky môže vyzerať zložito, PyTorch túto implementáciu skrýva vo vnútri triedy `LSTMCell` a poskytuje objekt `LSTM` na reprezentáciu celej LSTM vrstvy. Implementácia LSTM klasifikátora bude preto veľmi podobná jednoduchému RNN, ktorý sme videli vyššie:


In [4]:
class LSTMClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_class):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.embedding = torch.nn.Embedding(vocab_size, embed_dim)
        self.embedding.weight.data = torch.randn_like(self.embedding.weight.data)-0.5
        self.rnn = torch.nn.LSTM(embed_dim,hidden_dim,batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, num_class)

    def forward(self, x):
        batch_size = x.size(0)
        x = self.embedding(x)
        x,(h,c) = self.rnn(x)
        return self.fc(h[-1])

In [5]:
net = LSTMClassifier(vocab_size,64,32,len(classes)).to(device)
train_epoch(net,train_loader, lr=0.001)

3200: acc=0.259375
6400: acc=0.25859375
9600: acc=0.26177083333333334
12800: acc=0.2784375
16000: acc=0.313
19200: acc=0.3528645833333333
22400: acc=0.3965625
25600: acc=0.4385546875
28800: acc=0.4752777777777778
32000: acc=0.505375
35200: acc=0.5326704545454546
38400: acc=0.5557552083333334
41600: acc=0.5760817307692307
44800: acc=0.5954910714285714
48000: acc=0.6118333333333333
51200: acc=0.62681640625
54400: acc=0.6404779411764706
57600: acc=0.6520138888888889
60800: acc=0.662828947368421
64000: acc=0.673546875
67200: acc=0.6831547619047619
70400: acc=0.6917897727272727
73600: acc=0.6997146739130434
76800: acc=0.707109375
80000: acc=0.714075
83200: acc=0.7209134615384616
86400: acc=0.727037037037037
89600: acc=0.7326674107142858
92800: acc=0.7379633620689655
96000: acc=0.7433645833333333
99200: acc=0.7479032258064516
102400: acc=0.752119140625
105600: acc=0.7562405303030303
108800: acc=0.76015625
112000: acc=0.7641339285714286
115200: acc=0.7677777777777778
118400: acc=0.77112331081

(0.03487814127604167, 0.7728)

## Zbalené sekvencie

V našom príklade sme museli doplniť všetky sekvencie v minibatch nulovými vektormi. Hoci to vedie k určitému plytvaniu pamäťou, pri RNN je ešte kritickejšie, že sa vytvárajú ďalšie RNN bunky pre doplnené položky vstupu, ktoré sa zúčastňujú tréningu, no nenesú žiadne dôležité vstupné informácie. Bolo by oveľa lepšie trénovať RNN iba na skutočnú veľkosť sekvencie.

Na tento účel je v PyTorch zavedený špeciálny formát ukladania doplnených sekvencií. Predpokladajme, že máme vstupný doplnený minibatch, ktorý vyzerá takto:
```
[[1,2,3,4,5],
 [6,7,8,0,0],
 [9,0,0,0,0]]
```
Tu 0 predstavuje doplnené hodnoty a skutočný vektor dĺžok vstupných sekvencií je `[5,3,1]`.

Aby sme mohli efektívne trénovať RNN s doplnenou sekvenciou, chceme začať tréning prvej skupiny RNN buniek s veľkým minibatch (`[1,6,9]`), ale potom ukončiť spracovanie tretej sekvencie a pokračovať v tréningu s kratšími minibatchmi (`[2,7]`, `[3,8]`) a tak ďalej. Takto je zbalená sekvencia reprezentovaná ako jeden vektor - v našom prípade `[1,6,9,2,7,3,8,4,5]`, a vektor dĺžok (`[5,3,1]`), z ktorého môžeme ľahko rekonštruovať pôvodný doplnený minibatch.

Na vytvorenie zbalenej sekvencie môžeme použiť funkciu `torch.nn.utils.rnn.pack_padded_sequence`. Všetky rekurentné vrstvy, vrátane RNN, LSTM a GRU, podporujú zbalené sekvencie ako vstup a produkujú zbalený výstup, ktorý je možné dekódovať pomocou `torch.nn.utils.rnn.pad_packed_sequence`.

Aby sme mohli vytvoriť zbalenú sekvenciu, musíme do siete odovzdať vektor dĺžok, a preto potrebujeme inú funkciu na prípravu minibatchov:


In [6]:
def pad_length(b):
    # build vectorized sequence
    v = [encode(x[1]) for x in b]
    # compute max length of a sequence in this minibatch and length sequence itself
    len_seq = list(map(len,v))
    l = max(len_seq)
    return ( # tuple of three tensors - labels, padded features, length sequence
        torch.LongTensor([t[0]-1 for t in b]),
        torch.stack([torch.nn.functional.pad(torch.tensor(t),(0,l-len(t)),mode='constant',value=0) for t in v]),
        torch.tensor(len_seq)
    )

train_loader_len = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=pad_length, shuffle=True)

Skutočná sieť by bola veľmi podobná `LSTMClassifier` uvedenému vyššie, ale `forward` prechod dostane ako vstup nielen vypočítanú dávku (padded minibatch), ale aj vektor dĺžok sekvencií. Po výpočte embeddingu vytvoríme zabalenú sekvenciu (packed sequence), prejdeme ju cez LSTM vrstvu a potom výsledok rozbalíme späť.

> **Note**: V skutočnosti nepoužívame rozbalený výsledok `x`, pretože vo výpočtoch, ktoré nasledujú, používame výstup zo skrytých vrstiev. Preto môžeme rozbaľovanie z tohto kódu úplne odstrániť. Dôvod, prečo ho tu uvádzame, je, aby ste mohli tento kód ľahko upraviť, ak by ste potrebovali použiť výstup siete v ďalších výpočtoch.


In [7]:
class LSTMPackClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_class):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.embedding = torch.nn.Embedding(vocab_size, embed_dim)
        self.embedding.weight.data = torch.randn_like(self.embedding.weight.data)-0.5
        self.rnn = torch.nn.LSTM(embed_dim,hidden_dim,batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, num_class)

    def forward(self, x, lengths):
        batch_size = x.size(0)
        x = self.embedding(x)
        pad_x = torch.nn.utils.rnn.pack_padded_sequence(x,lengths,batch_first=True,enforce_sorted=False)
        pad_x,(h,c) = self.rnn(pad_x)
        x, _ = torch.nn.utils.rnn.pad_packed_sequence(pad_x,batch_first=True)
        return self.fc(h[-1])

In [8]:
net = LSTMPackClassifier(vocab_size,64,32,len(classes)).to(device)
train_epoch_emb(net,train_loader_len, lr=0.001,use_pack_sequence=True)


3200: acc=0.285625
6400: acc=0.33359375
9600: acc=0.3876041666666667
12800: acc=0.44078125
16000: acc=0.4825
19200: acc=0.5235416666666667
22400: acc=0.5559821428571429
25600: acc=0.58609375
28800: acc=0.6116666666666667
32000: acc=0.63340625
35200: acc=0.6525284090909091
38400: acc=0.668515625
41600: acc=0.6822596153846154
44800: acc=0.6948214285714286
48000: acc=0.7052708333333333
51200: acc=0.71521484375
54400: acc=0.7239889705882353
57600: acc=0.7315277777777778
60800: acc=0.7388486842105263
64000: acc=0.74571875
67200: acc=0.7518303571428572
70400: acc=0.7576988636363636
73600: acc=0.7628940217391305
76800: acc=0.7681510416666667
80000: acc=0.7728125
83200: acc=0.7772235576923077
86400: acc=0.7815393518518519
89600: acc=0.7857700892857142
92800: acc=0.7895043103448276
96000: acc=0.7930520833333333
99200: acc=0.7959072580645161
102400: acc=0.798994140625
105600: acc=0.802064393939394
108800: acc=0.8051378676470589
112000: acc=0.8077857142857143
115200: acc=0.8104600694444445
118400

(0.029785829671223958, 0.8138166666666666)

> **Poznámka:** Môžete si všimnúť parameter `use_pack_sequence`, ktorý odovzdávame do tréningovej funkcie. Momentálne funkcia `pack_padded_sequence` vyžaduje, aby bol tensor dĺžky sekvencie na zariadení CPU, a preto tréningová funkcia musí zabrániť presunu dát o dĺžke sekvencie na GPU počas tréningu. Môžete sa pozrieť na implementáciu funkcie `train_emb` v súbore [`torchnlp.py`](../../../../../lessons/5-NLP/16-RNN/torchnlp.py).


## Obojsmerné a viacvrstvové RNN

V našich príkladoch všetky rekurentné siete fungovali jedným smerom, od začiatku sekvencie až po jej koniec. Vyzerá to prirodzene, pretože to pripomína spôsob, akým čítame a počúvame reč. Avšak, keďže v mnohých praktických prípadoch máme náhodný prístup k vstupnej sekvencii, môže dávať zmysel vykonávať rekurentné výpočty v oboch smeroch. Takéto siete sa nazývajú **obojsmerné** RNN a môžu byť vytvorené pridaním parametra `bidirectional=True` do konštruktora RNN/LSTM/GRU.

Pri práci s obojsmernou sieťou budeme potrebovať dva vektory skrytého stavu, jeden pre každý smer. PyTorch kóduje tieto vektory ako jeden vektor dvojnásobnej veľkosti, čo je veľmi praktické, pretože výsledný skrytý stav zvyčajne odovzdávate do plne prepojenej lineárnej vrstvy, a stačí len zohľadniť toto zvýšenie veľkosti pri vytváraní vrstvy.

Rekurentná sieť, či už jednosmerná alebo obojsmerná, zachytáva určité vzory v rámci sekvencie a môže ich uložiť do vektora stavu alebo odovzdať do výstupu. Rovnako ako pri konvolučných sieťach, môžeme na prvú vrstvu postaviť ďalšiu rekurentnú vrstvu, aby sme zachytili vzory na vyššej úrovni, ktoré sú vytvorené z nízkoúrovňových vzorov extrahovaných prvou vrstvou. To nás privádza k pojmu **viacvrstvová RNN**, ktorá pozostáva z dvoch alebo viacerých rekurentných sietí, kde výstup predchádzajúcej vrstvy je odovzdaný ako vstup do nasledujúcej vrstvy.

![Obrázok zobrazujúci viacvrstvovú LSTM-RNN](../../../../../translated_images/sk/multi-layer-lstm.dd975e29bb2a59fe.webp)

*Obrázok z [tohto skvelého článku](https://towardsdatascience.com/from-a-lstm-cell-to-a-multilayer-lstm-network-with-pytorch-2899eb5696f3) od Fernanda Lópeza*

PyTorch uľahčuje konštrukciu takýchto sietí, pretože stačí pridať parameter `num_layers` do konštruktora RNN/LSTM/GRU, aby sa automaticky vytvorilo niekoľko vrstiev rekurencie. To tiež znamená, že veľkosť vektora skrytého stavu sa úmerne zvýši, a budete musieť toto zvýšenie zohľadniť pri práci s výstupom rekurentných vrstiev.


## RNN pre iné úlohy

V tejto jednotke sme videli, že RNN môžu byť použité na klasifikáciu sekvencií, ale v skutočnosti dokážu zvládnuť oveľa viac úloh, ako je generovanie textu, strojový preklad a ďalšie. Týmto úlohám sa budeme venovať v nasledujúcej jednotke.



---

**Upozornenie**:  
Tento dokument bol preložený pomocou služby AI prekladu [Co-op Translator](https://github.com/Azure/co-op-translator). Aj keď sa snažíme o presnosť, prosím, berte na vedomie, že automatizované preklady môžu obsahovať chyby alebo nepresnosti. Pôvodný dokument v jeho rodnom jazyku by mal byť považovaný za autoritatívny zdroj. Pre kritické informácie sa odporúča profesionálny ľudský preklad. Nie sme zodpovední za akékoľvek nedorozumenia alebo nesprávne interpretácie vyplývajúce z použitia tohto prekladu.
